In [ ]:
import datetime
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [ ]:
now = datetime.datetime.now

In [ ]:
now()

datetime.datetime(2023, 7, 12, 16, 44, 3, 726321)

In [ ]:
#HYPERPARAMETERS INITIALISATION

batch_size = 128
num_classes = 5
epochs = 5

In [ ]:
#PARAMETERS INITIALISATION FOR CNN-ARCHITECTURE

img_rows, img_cols = 28,28
filters = 32
pool_size = 2
kernel_size = 3

In [ ]:
#THIS HANDLES SOME VARIABILITY IN HOW INPUT DATA IS LOCATED

if K.image_data_format() == 'channels_first':
  input_shape = (1 , img_rows,img_cols)
else:
  input_shape = (img_rows,img_cols , 1)

In [ ]:
#to simplify things, a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which are training or fine-tuning the layers

def train_model(model , train, test , num_classes):
  x_train = train[0].reshape((train[0].shape[0],) + input_shape)
  x_test = test[0].reshape((test[0].shape[0] ,) + input_shape)
  x_train = x_train.astype("float32")
  x_test = x_test.astype('float32')
  x_train /= 255 # scalingg every pixel value less than 1 and gaeter than 0
  x_test /= 255
  print("x_train shape:" , x_train.shape)
  print(x_train.shape[0] , 'train samples')
  print(x_test.shape[0] , 'test samples')

  # convert class vectors into binary classification or one-hot encoding

  y_train = keras.utils.to_categorical(train[1] , num_classes)
  y_test = keras.utils.to_categorical(test[1] , num_classes)

  model.compile(loss = 'categorical_crossentropy' , optimizer = 'adadelta' , metrics = ["accuracy"]) # here adaadelta used instead of Adam or RMSprop to slow or smoothen process of finding optimal weights

  t = now()
  model.fit(x_train , y_train , batch_size = batch_size , epochs =  epochs , verbose = 1 , validation_data = (x_test,y_test) )
  print("Training time is " , (now() - t))

  score = model.evaluate(x_test , y_test , verbose = 1)
  print("Test score : " , score[0])
  print("Test accuracy : " , score[1])



In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#create two datasets: one with digits below 5 and one with 5 and above

x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5
x_test_gt5 = x_test[y_test >= 5]
y_test_gt5 = y_test[y_test >= 5] - 5



In [ ]:
 #defining a "early features layers" which are freezed from fine-tuning and will be
 #used for Transfer Learning for other model's architecture

feature_layers = [
    Conv2D(filters , kernel_size , padding = 'valid' , input_shape = input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size = pool_size),
    Dropout(0.25),
    Flatten(),
]

In [ ]:
# defining fine-tuning layers or outer_domain of CNN layers

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [ ]:
# creating sequential models by combining two above layers into single

model = Sequential(feature_layers + classification_layers)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [ ]:
#training the model to identify digits 5,6,7,8,9

train_model(model,
            (x_train_gt5 , y_train_gt5),
            (x_test_gt5 , y_test_gt5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 62s 264ms/step - loss: 1.5984 - accuracy: 0.2465 - val_loss: 1.5816 - val_accuracy: 0.3972
Epoch 2/5
230/230 [==============================] - 60s 259ms/step - loss: 1.5775 - accuracy: 0.3124 - val_loss: 1.5566 - val_accuracy: 0.5419
Epoch 3/5
230/230 [==============================] - 63s 275ms/step - loss: 1.5538 - accuracy: 0.3821 - val_loss: 1.5294 - val_accuracy: 0.6260
Epoch 4/5
230/230 [==============================] - 60s 262ms/step - loss: 1.5272 - accuracy: 0.4487 - val_loss: 1.4993 - val_accuracy: 0.6632
Epoch 5/5
230/230 [==============================] - 61s 263ms/step - loss: 1.5002 - accuracy: 0.4909 - val_loss: 1.4658 - val_accuracy: 0.6780
Training time is  0:05:05.928885
152/152 [==============================] - 2s 15ms/step - loss: 1.4658 - accuracy: 0.6780
Test score :  1.4658153057098389
Test accuracy :  0.6780498027801514


In [ ]:
#FREEZING EARLY LAYERS

for i in feature_layers:
  i.trainable = False

In [ ]:
model.summary() #here training params less than earlier due to save of traing of early layers which is not neede for next task

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [ ]:
train_model(model , (x_train_lt5,y_train_lt5),(x_test_lt5,y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 21s 85ms/step - loss: 1.6142 - accuracy: 0.2213 - val_loss: 1.5909 - val_accuracy: 0.3069
Epoch 2/5
240/240 [==============================] - 21s 89ms/step - loss: 1.5778 - accuracy: 0.2922 - val_loss: 1.5519 - val_accuracy: 0.4271
Epoch 3/5
240/240 [==============================] - 20s 84ms/step - loss: 1.5433 - accuracy: 0.3733 - val_loss: 1.5137 - val_accuracy: 0.5238
Epoch 4/5
240/240 [==============================] - 20s 82ms/step - loss: 1.5096 - accuracy: 0.4461 - val_loss: 1.4761 - val_accuracy: 0.6104
Epoch 5/5
240/240 [==============================] - 21s 87ms/step - loss: 1.4762 - accuracy: 0.5153 - val_loss: 1.4388 - val_accuracy: 0.7494
Training time is  0:02:22.697417
161/161 [==============================] - 3s 18ms/step - loss: 1.4388 - accuracy: 0.7494
Test score :  1.438792109489441
Test accuracy :  0.7493675947189331
